In [2]:
!pip install tensorflow==2.18
!pip install --upgrade pip
!pip install --upgrade tensorflow tensorflow_datasets

  Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached ml_dtypes-0.4.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
Using cached tensorflow-2.18.0-cp311-cp311-win_amd64.whl (7.5 kB)
Using cached ml_dtypes-0.4.1-cp311-cp311-win_amd64.whl (126 kB)
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)

  Attempting uninstall: ml-dtypes

    Found existing installation: ml_dtypes 0.5.4

    Uninstalling ml_dtypes-0.5.4:

      Successfully uninstalled ml_dtypes-0.5.4

  Attempting uninstall: tensorboard

    Found existing installation: tensorboard 2.20.0

   ------------- -------------------------- 1/3 [tensorboard]
    Uninstalling tensorboard-2.20.0:
   ------------- -------------------------- 1/3 [tensorboard]
   ------------- -------------------------- 1/3 [tensorboard]
      Successfully uninstalled tensorboard-2.20.0
   ------------- -------------------------- 1/3 [tensorboard

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.4 which is incompatible.
tensorflow-intel 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.20.0 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
from sklearn.preprocessing import StandardScaler

#code for feature engineering
def featurize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(2050)

    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd = Descriptors.NumHDonors(mol)
    hba = Descriptors.NumHAcceptors(mol)
    tpsa = Descriptors.TPSA(mol)

    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
    fp = np.array(fp)

    return np.concatenate([[mw, logp, hbd, hba, tpsa], fp])
train = pd.read_csv("/Users/hwind/Downloads/Python/Master/3_Semester/Data Science/Project/ids2025_group7_project_g1/Data/train.csv")
test = pd.read_csv("/Users/hwind/Downloads/Python/Master/3_Semester/Data Science/Project/ids2025_group7_project_g1/Data/test.csv")

X_train = np.vstack(train["SMILES"].apply(featurize_smiles)).astype('float32')
X_test = np.vstack(test["SMILES"].apply(featurize_smiles)).astype('float32')
y_train = train["Tm"].values.astype("float32")


C:\Users\hwind\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
num_features = X_train.shape[1]
batch_size = 128
ds_train = tf.data.Dataset.from_tensor_slices((X_train.astype('float32'), y_train.astype('float32')))
ds_train = ds_train.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

n_hidden_1 = 256
n_hidden_2 = 128
learning_rate = 0.001

initializer = tf.initializers.GlorotUniform()

weights = {
    "h1": tf.Variable(initializer([num_features, n_hidden_1], dtype=tf.float32)),
    "h2": tf.Variable(initializer([n_hidden_1, n_hidden_2], dtype=tf.float32)),
    "out": tf.Variable(initializer([n_hidden_2, 1], dtype=tf.float32))
}

biases = {
    "b1": tf.Variable(tf.zeros([n_hidden_1], dtype=tf.float32)),
    "b2": tf.Variable(tf.zeros([n_hidden_2], dtype=tf.float32)),
    "out": tf.Variable(tf.zeros([1], dtype=tf.float32))
}

#Model
def neural_net(x):
    layer1 = tf.nn.relu(tf.matmul(x, weights["h1"]) + biases["b1"])
    layer2 = tf.nn.relu(tf.matmul(layer1, weights["h2"]) + biases["b2"])
    out = tf.matmul(layer2, weights["out"]) + biases["out"]  # LINEAR OUTPUT
    return out  # shape: (batch, 1)

#Loss + Optimizer
def mse_loss(y_pred, y_true):
    y_true = tf.reshape(y_true, (-1, 1))
    return tf.reduce_mean((y_true - y_pred) ** 2)

optimizer = tf.optimizers.Adam(learning_rate)

#print("x dtype:", bx.dtype)
#print("weights dtype:", weights["h1"].dtype)

#Training
def run_optimization(x, y):
    with tf.GradientTape() as tape:
        pred = neural_net(x)
        loss = mse_loss(pred, y)

    trainable_vars = list(weights.values()) + list(biases.values())
    gradients = tape.gradient(loss, trainable_vars)
    optimizer.apply_gradients(zip(gradients, trainable_vars))
    return loss

epochs = 20

for epoch in range(1, epochs + 1):
    batch_losses = []
    for bx, by in ds_train:
        loss = run_optimization(bx, by)
        batch_losses.append(loss.numpy())

    print(f"Epoch {epoch:02d}  Loss: {np.mean(batch_losses):.4f}")

# prediction
pred_test = neural_net(X_test).numpy().flatten()

print("\nexample_prediction:", pred_test[:5])
submission = pd.DataFrame({
    "id": test["id"],
    "Tm": pred_test
})

#submission.to_csv("Submissions/predictions_NN.csv", index=False)



Epoch 01  Loss: 70523.1641
Epoch 02  Loss: 24247.8750
Epoch 03  Loss: 9856.8311
Epoch 04  Loss: 8316.0889
Epoch 05  Loss: 7692.0044
Epoch 06  Loss: 7235.2231
Epoch 07  Loss: 6825.7871
Epoch 08  Loss: 6354.2930
Epoch 09  Loss: 5923.7734
Epoch 10  Loss: 5477.4316
Epoch 11  Loss: 5074.1577
Epoch 12  Loss: 4667.1147
Epoch 13  Loss: 4386.2671
Epoch 14  Loss: 4041.6868
Epoch 15  Loss: 3838.8848
Epoch 16  Loss: 3585.7917
Epoch 17  Loss: 3499.6199
Epoch 18  Loss: 3311.0105
Epoch 19  Loss: 3050.7771
Epoch 20  Loss: 2930.0435

example_prediction: [333.5938  384.3297  181.126   197.14261 244.22209]


In [6]:
#best try
#TensorFlow Dataset
batch_size = 128
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train.astype("float32")))
ds_train = ds_train.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

#better model
num_features = X_train.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(num_features,)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1)  # linear output
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse"
)

#training
callback = tf.keras.callbacks.EarlyStopping(
    patience=30,
    restore_best_weights=True
)

history = model.fit(
    ds_train,
    epochs=300,
    callbacks=[callback],
    verbose=1
)

# Predictions
pred_test = model.predict(X_test).flatten()

pd.DataFrame({
    "id": test["id"],
    "Tm": pred_test
})#.to_csv("Submissions/predictions_NN_improved.csv", index=False)


Epoch 1/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 79415.0078
Epoch 2/300
 4/21 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 68616.3867 

C:\Users\hwind\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 66682.3281
Epoch 3/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 45194.4844
Epoch 4/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 23276.1035
Epoch 5/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 9299.9072 
Epoch 6/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 6281.6230
Epoch 7/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 4951.7944
Epoch 8/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 3439.5146
Epoch 9/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2936.0452
Epoch 10/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2540.1931
Epoch 11/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1802.6133
Epoch 12/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1908.3459
Epoch 13/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2053.9492
Epoch 14/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1732.8793
Epoch 15/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1620.0907
Epoch 16/300
21/21 ━━━

,id,Tm
0,1022,344.937775
1,1146,256.665710
2,79,208.942673
3,2279,220.955521
4,1342,250.004120
...,...,...
661,2663,276.214050
662,624,220.769348
663,2655,194.528580
664,2089,255.882507


In [ ]:
#optimization of the code above
#add dropout, cosine LR Decay, layer setup, early stopping, 5 times ensemble

#massive overfitting !!!!!
batch_size = 128

# Validation Split (15%)
val_split = int(len(X_train) * 0.15)
X_val = X_train[:val_split]
y_val = y_train[:val_split]
X_tr = X_train[val_split:]
y_tr = y_train[val_split:]

ds_train = tf.data.Dataset.from_tensor_slices((X_tr, y_tr.astype("float32")))
ds_train = ds_train.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

ds_val = tf.data.Dataset.from_tensor_slices((X_val, y_val.astype("float32")))
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)

num_features = X_train.shape[1]


#Cosine LR Decay
def get_optimizer():
    lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(
        initial_learning_rate=1e-3,
        first_decay_steps=2500,
        t_mul=1.5,
        m_mul=0.9,
        alpha=1e-4,
    )
    return tf.keras.optimizers.Adam(lr_schedule)


#Modell-Builder
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(num_features,)),

        tf.keras.layers.Dense(1024, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.15),

        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),

        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dropout(0.05),

        tf.keras.layers.Dense(128, activation="relu"),

        tf.keras.layers.Dense(1)
    ])

    model.compile(
        optimizer=get_optimizer(),
        loss="mse"
    )

    return model


#training
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=40,
    restore_best_weights=True
)


# Ensemble (5 Models)
all_preds = []

for seed in [0, 1, 2, 3, 4]:
    print(f"Training Modell with Seed {seed}")

    tf.random.set_seed(seed)
    np.random.seed(seed)

    model = build_model()

    model.fit(
        ds_train,
        validation_data=ds_val,
        epochs=400,
        verbose=1,
        callbacks=[callback]
    )

    preds = model.predict(X_test, batch_size=1024).flatten()
    all_preds.append(preds)

# mean of Predictions
final_pred = np.mean(all_preds, axis=0)


sub = pd.DataFrame({
    "id": test["id"],
    "Tm": final_pred
})

#sub.to_csv("Submissions/predictions_NN_ensemble.csv", index=False)




🔵 Training Modell mit Seed 0
Epoch 1/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - loss: 71650.3438 - val_loss: 27928.5820
Epoch 2/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 21966.4473 - val_loss: 8225.7783
Epoch 3/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 4305.8760 - val_loss: 9724.8135
Epoch 4/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 3081.9299 - val_loss: 6910.1196
Epoch 5/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 2807.1604 - val_loss: 10249.9863
Epoch 6/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 2316.7400 - val_loss: 16139.3545
Epoch 7/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 2208.0989 - val_loss: 14996.2520
Epoch 8/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 1892.7788 - val_loss: 12685.8613
Epoch 9/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 1779.5704 - val_loss: 14434.3643
Epoch 10/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 1740.8087 - val_loss: 14330.3457
Epoch 11/400
18/18 ━━━━━━━━━━━━━━━━━━━━ 

In [8]:
#not as good as the second approach
batch_size = 128
val_split = int(len(X_train) * 0.15)

X_val = X_train[:val_split]
y_val = y_train[:val_split]
X_tr = X_train[val_split:]
y_tr = y_train[val_split:]

ds_train = tf.data.Dataset.from_tensor_slices((X_tr, y_tr.astype("float32")))
ds_train = ds_train.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

ds_val = tf.data.Dataset.from_tensor_slices((X_val, y_val.astype("float32")))
ds_val = ds_val.batch(batch_size).prefetch(tf.data.AUTOTUNE)


# Anti-Overfitting Model
def build_model():
    reg = tf.keras.regularizers.l2(1e-5)

    model = tf.keras.Sequential([
        tf.keras.layers.Input((X_train.shape[1],)),

        tf.keras.layers.Dense(256, activation="relu", kernel_regularizer=reg),
        tf.keras.layers.Dropout(0.30),

        tf.keras.layers.Dense(128, activation="relu", kernel_regularizer=reg),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=reg),
        tf.keras.layers.Dropout(0.20),

        tf.keras.layers.Dense(32, activation="relu", kernel_regularizer=reg),

        tf.keras.layers.Dense(1)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss="mse"
    )

    return model


# Training
callback = tf.keras.callbacks.EarlyStopping(
    patience=40,
    restore_best_weights=True,
    monitor="val_loss"
)

model = build_model()

history = model.fit(
    ds_train,
    epochs=300,
    validation_data=ds_val,
    callbacks=[callback],
    verbose=1
)

# Prediction
pred = model.predict(X_test).flatten()

pd.DataFrame({
    "id": test["id"],
    "Tm": pred
})#.to_csv("Submissions/predictions_NN_regularized.csv", index=False)


Epoch 1/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 51810.1133 - val_loss: 10175.5322
Epoch 2/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 13745.8955 - val_loss: 9917.3125
Epoch 3/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 10854.4033 - val_loss: 7695.5195
Epoch 4/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 9744.0000 - val_loss: 7091.2080
Epoch 5/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8893.9043 - val_loss: 6522.5488
Epoch 6/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8198.1670 - val_loss: 5828.9263
Epoch 7/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8035.3452 - val_loss: 5510.0679
Epoch 8/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 7419.2622 - val_loss: 5596.1387
Epoch 9/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6593.2725 - val_loss: 4522.7803
Epoch 10/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6137.7695 - val_loss: 4688.0723
Epoch 11/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5442.3335 - va

,id,Tm
0,1022,357.531769
1,1146,346.450684
2,79,206.580292
3,2279,177.256149
4,1342,236.501114
...,...,...
661,2663,313.843353
662,624,301.870972
663,2655,229.336014
664,2089,262.117554


In [9]:
# usage of optuna
#is worse than the second try
from sklearn.model_selection import train_test_split
#Data

X_train_np = X_train.astype("float32")
y_train_np = y_train.astype("float32")

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train_np, y_train_np, test_size=0.15, random_state=42
)

# TF Datasets Funktion
def make_dataset(X, y, batch_size):
    return (
        tf.data.Dataset.from_tensor_slices((X, y))
        .shuffle(len(X))
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

# Optuna 
def objective(trial):

    # Hyperparameter search
    n1 = trial.suggest_int("n_hidden_1", 128, 1024)
    n2 = trial.suggest_int("n_hidden_2", 64, 512)
    n3 = trial.suggest_int("n_hidden_3", 0, 256)   # 0 = kein Layer 3

    lr = trial.suggest_float("learning_rate", 1e-4, 5e-3, log=True)
    dropout = trial.suggest_float("dropout", 0.0, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256, 512])
    activation = trial.suggest_categorical("activation", ["relu", "selu", "gelu"])

    # Dataset
    ds_train = make_dataset(X_tr, y_tr, batch_size)
    ds_val = make_dataset(X_val, y_val, batch_size)

    # Model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_tr.shape[1],)))
    model.add(tf.keras.layers.Dense(n1, activation=activation))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.keras.layers.Dense(n2, activation=activation))

    if n3 > 0:
        model.add(tf.keras.layers.Dense(n3, activation=activation))

    model.add(tf.keras.layers.Dense(1))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr),
        loss="mse"
    )

    # Training 
    history = model.fit(
        ds_train,
        validation_data=ds_val,
        epochs=40,        # kurz halten, wird oft ausgeführt
        verbose=0
    )

    # goal: Minimal Val-MSE
    val_loss = min(history.history["val_loss"])
    return val_loss

# Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=40)   # 40–80 empfohlen

print("\nBest Trial:")
print(study.best_trial.params)

# FINAL MODEL with best parameters
best_params = study.best_trial.params

n1 = best_params["n_hidden_1"]
n2 = best_params["n_hidden_2"]
n3 = best_params["n_hidden_3"]
lr = best_params["learning_rate"]
dropout = best_params["dropout"]
batch_size = best_params["batch_size"]
activation = best_params["activation"]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=(X_train_np.shape[1],)))
model.add(tf.keras.layers.Dense(n1, activation=activation))
model.add(tf.keras.layers.Dropout(dropout))
model.add(tf.keras.layers.Dense(n2, activation=activation))
if n3 > 0:
    model.add(tf.keras.layers.Dense(n3, activation=activation))
model.add(tf.keras.layers.Dense(1))

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr),
    loss="mse"
)

ds_full = make_dataset(X_train_np, y_train_np, batch_size)

# full training
history = model.fit(
    ds_full,
    epochs=200,
    verbose=1
)

#Prediction
pred_test = model.predict(X_test.astype("float32")).flatten()

pd.DataFrame({
    "id": test["id"],
    "Tm": pred_test
})#.to_csv("Submissions/predictions_NN_optuna.csv", index=False)




Best Trial:
{'n_hidden_1': 315, 'n_hidden_2': 424, 'n_hidden_3': 205, 'learning_rate': 0.0006846199681208632, 'dropout': 0.14641949270018959, 'batch_size': 64, 'activation': 'gelu'}
Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 24961.8086
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 7865.4448
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 5741.5264
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 4138.1787
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 3454.4573
Epoch 6/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 3025.1982
Epoch 7/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 2593.9482
Epoch 8/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2410.1074
Epoch 9/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2145.7219
Epoch 10/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 2071.3965
Epoch 11/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1961.5610
Epoch 12/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 1

,id,Tm
0,1022,340.564850
1,1146,265.474060
2,79,227.070847
3,2279,209.710571
4,1342,239.315216
...,...,...
661,2663,277.880676
662,624,283.993744
663,2655,232.423004
664,2089,228.327103


In [12]:
#best try without feature engineering
train = pd.read_csv("/Users/hwind/Downloads/Python/Master/3_Semester/Data Science/Project/ids2025_group7_project_g1/Data/train.csv")
test = pd.read_csv("/Users/hwind/Downloads/Python/Master/3_Semester/Data Science/Project/ids2025_group7_project_g1/Data/test.csv")
groups =[col for col in train.columns if col.startswith('Group')]
print(train.head())
X_train = train[groups]
y_train = train['Tm']
X_test = test[groups]


batch_size = 128
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train.astype("float32")))
ds_train = ds_train.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

num_features = X_train.shape[1]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(num_features,)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1)  # linear output
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse"
)

callback = tf.keras.callbacks.EarlyStopping(
    patience=30,
    restore_best_weights=True
)

history = model.fit(
    ds_train,
    epochs=300,
    callbacks=[callback],
    verbose=1
)

pred_test = model.predict(X_test).flatten()

pd.DataFrame({
    "id": test["id"],
    "Tm": pred_test
}).to_csv("/Users/hwind/Downloads/Python/Master/3_Semester/Data Science/Project/ids2025_group7_project_g1/Submissions/predictions_NN_wo_feature_engineering.csv", index=False)


     id                       SMILES      Tm  Group 1  Group 2  Group 3  \
0  2175        FC1=C(F)C(F)(F)C1(F)F  213.15        0        0        0   
1  1222  c1ccc2c(c1)ccc3Nc4ccccc4c23  407.15        0        0        0   
2  2994          CCN1C(C)=Nc2ccccc12  324.15        2        1        0   
3  1704                   CC#CC(=O)O  351.15        1        0        0   
4  2526                    CCCCC(S)C  126.15        2        3        0   

   Group 4  Group 5  Group 6  Group 7  ...  Group 415  Group 416  Group 417  \
0        0        0        0        0  ...          0          0          0   
1        0        0        0        0  ...          0          0          0   
2        0        0        0        0  ...          0          0          0   
3        0        0        0        0  ...          0          0          0   
4        0        0        0        0  ...          0          0          0   

   Group 418  Group 419  Group 420  Group 421  Group 422  Group 423  Group

C:\Users\hwind\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 47625.6641
Epoch 4/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 24744.9746
Epoch 5/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 9116.2197 
Epoch 6/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3465.7000
Epoch 7/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2354.4543
Epoch 8/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2660.3381
Epoch 9/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2316.0029
Epoch 10/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2808.6179
Epoch 11/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2360.6787
Epoch 12/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1658.4675
Epoch 13/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 2046.4220
Epoch 14/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1976.5376
Epoch 15/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1622.5945
Epoch 16/300
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1825.8505
Epoch 17/300
21/21 ━━━━━━━━━━━━━━━━━

In [13]:
#first try neural networks without the part of the lecture
#neural network
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV


param_grid_mlp = {
    'hidden_layer_sizes': [(64,64), (128,64)],
    'activation': ['relu'],
    'alpha': [0.0001, 0.001],       # Regularisierung
    'learning_rate': ['adaptive']
}

#mlp = MLPRegressor(max_iter=300)
mlp = MLPRegressor(max_iter=500, early_stopping=True, random_state=0)

grid_mlp = GridSearchCV(mlp, param_grid_mlp, cv=2, scoring="neg_mean_absolute_error",n_jobs=-1 )
grid_mlp.fit(X_train, y_train)

print("Beste Parameter MLP:", grid_mlp.best_params_)
print("Bester Score:", -grid_mlp.best_score_)

y_pred = grid_mlp.predict(X_test)

submission = pd.DataFrame({
    'id': test['id'],
    'Tm': y_pred})
#submission.to_csv("Submissions/predictions_neural_network_fast.csv", index=False)



Beste Parameter MLP: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (64, 64), 'learning_rate': 'adaptive'}
Bester Score: 39.12846646924251
